## <span style="color: Gold"> RAG

In [ ]:
# RAG
# 우리회사의 복지제도는?
# LLM은 학습데이터에 없는 최신/특정 정보를 모름
# ==> 이를 RAG로 해결함

# RAG 해결책 (Retrieval Augmented Generation 검색 기반 생성) = 검색 + 생성
# “모델이 외부 문서에서 필요한 정보를 검색해 참고한 뒤, 그 정보를 바탕으로 답변을 생성하는 기술”
# 질문(query) → 벡터화 → 문서 벡터와 유사도 계산 → 관련 문서(top-k) 추출 → LLM으로 답 생성
# 1. 회사 문서에서 관련 정보 검색
# 2. 검색된 정보를 LLM에게 컨텍스트로 제공
# 3. LLM이 컨텍스 기반으로 정확한 답변 생성

# [준비 단계]
# 문서들 --> 청크(chunk)로 분할 --> 인덱스 생성(벡터로 변환) --> 벡터DB 저장
    # chunk : 문서를 쪼갠 작은 조각, 긴 문서도 여러 청크로 나누어 검색, 검색시 유사 문서(chunk)만 모델에게 전달 --> 토큰비용 절약
    # index : chunk를 벡터화해서 저장한 자료 구조

# [쿼리 단계]
# 질문 --> 벡터변환 --> 유사도 검색 --> 상위 k개 선택 --> 컨텍스트+질문 --> LLM --> 답변

In [ ]:
# %pip install llama-index
# %pip install llama-index-embeddings-huggingface


In [ ]:
# llama_index 
# llama_index : RAG 구축용 프레임워크
# 문서로딩 : 다양한 형태(PDF, 웹페이지, 폴더, Notion 등) 읽기
# 문서 청크 : 문서를 적절한 길이로 나누기
# 임베딩 생성 :	텍스트 → 벡터 변환(openAI, HF 등 선택 가능)
# 벡터 스토어 저장 :	Chroma, Pinecone, FAISS 등과 자동 연결
# 검색 / 쿼리엔진 :	가장 유사한 문서 검색
# RAG 파이프라인 구성 :	검색 + 답변 생성(end-to-end RAG)
# 즉, RAG 아키텍처를 자동으로 만들어 주는 종합 툴이 바로 llama_index.

# 문서를 자르고 --> 임베딩을 생성하고 --> 벡터DB에 저장 --> 검색 --> LLM에게 전달해서 답변 생성
    # index = VectorStoreIndex.from_documents(documents)
    # query_engine = index.as_query_engine()     
    # response = query_engine.query("대한민국의 수도는?")






from llama_index.core import Document, VectorStoreIndex

import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
client = OpenAI()


# 1단계. 문서 준비
document = [
    Document(text='대한민국의 수도는 서울입니다.'),
    Document(text='프랑스의 수도는 파리 입니다.')
]

# 2단계. 인덱스 생성 (자동으로 벡터화) VectorStoreIndex.from_documents : "문서--> 벡터화--> 저장" 과정의 결과물
    # 각 문서를 청크로 나눔 
    # 각 청크를 벡터(숫자배열)로 변환 --> 임베딩 생성 , openAI에서 제공하는 embedding 모델을 사용해서 벡터화
    # 벡터를 인메모리 벡터 스토어에 저장 --> 빠른 유사도 검색 가능
index = VectorStoreIndex.from_documents(document,
                                        # embed_model="local"  # embed_model="local" 로컬 임베딩 사용 / 혹은 상기 openai API 사용하기
                                        )   

# 3단계. Query Engine 생성
    # 사용자가 질문(query)을 던지면 인덱스(vectorstoreindex) 안에서 질문과 가장 유사한 청크를 찾아냄
    # 상위 k개 청크를 LLM에게 전달 --> 답 생성
    # 역할 : "검색 + 생성 연결고리"
query_engine = index.as_query_engine(similarity_top_k=2)  # similarity_top_k=2 : 질문(query)과 가장 유사한 청크 2개만 가져와!

# 4단계. 쿼리 실행  -------------> 문서에서 답변을 찾아냄
    # 질문 --> 유사청크검색 --> 답생성
response = query_engine.query('대한민국의 수도는 어디입니까?')
print(response)

서울


In [ ]:
# Chunk 청크
# 문서검색의 최소 단위
# 모델이 한번에 처리할 수 있는 길이로 잘라낸 텍스트
# 사용하는 이유 : 모델에 입력할 수 있는 길이가 제한되어 있고, 문서가 길면 한번에 처리할 수 없어서 청크로 나눠서 처리함
# 벡터 DB에서 문서전체가 아니라 청크단위로 벡터화
# 질문과 유사한 작은 단위를 찾아 답변을 생성
# 전체 문서를 이해하는 대신 청크별로 처리해서 중요한 부분에 집중


# 청크 크기를 작은 조각으로 쪼개면, 질문과 가장 관련된 작은 부분만 정확하게 매칭 --> 검색 정밀도 높아짐 -->개수가 많으면 임베딩해야하는 횟수가 늘어남(API)
# 청크 크기를 큰 조각으로 쪼개면, 넓게 검색 --> 검색 정확도 낮아짐 --> 임베딩 해야하는 횟수 (API 호출)가 작음 --> 비용 감소
   # 예시
    # 문서1: 서울 이야기
    # 문서2: 파리 이야기
        # 프랑스 수도가 어디ㅑ?
        # 청크사이즈가 100이면, 파리관련 내용만 담긴 작은 청크가 매칭
        # 청크사이즈가 2000이면, 서울+파리+다른 내용까지 섞임

    # 문서 10,000토큰
        # chunk_size 100 → 청크 100개 → 임베딩 100번 호출
        # chunk_size 500 → 청크 20개 → 임베딩 20번 호출




from llama_index.core import Settings
Settings.chunk_size = 512 # 512가 기본값
# Settings.chunk_overlap = 128 # 128이 기본값
# overlap 청크 간 겹침을 나타냄 --> 겹침 --> 문맥 유지 가능?
Settings.chunk_overlap = 50

# 유사도 임계값 설정
    # similarity_top_k 값이 너무 작으면 필요한 정보가 누락될 수 있음
    # 너무 크면 불필요한 청크까지 넣어서 토큰 비용 증가
    # 보통 2~5 정도로 설정해서 실험하는 게 일반적
from llama_index.core.postprocessor import SimilarityPostprocessor
qeury_engine = index.as_query_engine(
    similarity_top_k = 2,  # 유사도 상위 2
    node_posprocessors =[
        SimilarityPostprocessor(similarity_cutoff=0.7)  #유사도 0.7 미만의 문서는 제외 (노이즈 제거)
    ]
)

# 배치 처리
Settings.embed_batch_size = 100

## <span style="color: Gold"> 한국어 데이터 RAG 구현

In [ ]:
# 한국어 데이터로 RAG 구현
from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# 1. LLM 임베딩 모델 설정 (벡터화를 담당하는 모델이 임베딩 모델) (한국어 데이터를 위해 임베딩 모델을 설정)
# index = VectorStoreIndex.from_documents(document)  ---> 이건 LlamaIndex가 내부적으로 기본 OpenAI 임베딩 모델을 자동으로 사용함
Settings.llm = OpenAI(model='gpt-4o-mini', temperature=0.1)
Settings.embed_model = OpenAIEmbedding(model='text-embedding-3-small')


In [ ]:
# 2. 문서준비
documents = [
    Document(
        text="김치는 한국의 대표적인 발효 음식입니다. 배추에 고춧가루, 마늘, 생강 등을 넣어 만듭니다.",
        metadata={"source": "한국 음식 백과", "category": "반찬"}
    ),
    Document(
        text="비빔밥은 밥 위에 여러 가지 나물과 고기, 계란을 올려 고추장과 섞어 먹는 음식입니다.",
        metadata={"source": "한국 음식 백과", "category": "밥 요리"}
    ),
    Document(
        text="불고기는 양념한 소고기를 구워 먹는 한국의 전통 음식입니다. 달콤하고 짭짤한 맛이 특징입니다.",
        metadata={"source": "한국 음식 백과", "category": "고기 요리"}
    ),
    Document(
        text="떡볶이는 가래떡에 고추장 양념을 넣어 볶은 한국의 길거리 음식입니다. 달콤하고 매운 맛이 특징입니다.",
        metadata={"source": "한국 음식 백과", "category": "분식"}
    ),
]

In [16]:
# 3. 벡터 인덱스 생성
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# 4. 쿼리 엔진 생성
query_engine = index.as_query_engine (
    similarity_top_k = 2,
    # node_postprocessors = [SimilarityPostprocessor(similarity_cutoff=0.7)] # 데이터가 적거나 문장 표현이 다양하면 cutoff 실행안하는게 나음
)

In [ ]:
# 5. 질문하기
questions = [
    '김치는 어떤 음식인가요?',
    '비빔밥을 어떻게 먹나요?',
    '한국의 고기 요리에는 뭐가 있나요?'
]

for q in questions:
    response = query_engine.query(q)
    print(f'질문 : {q}\n답변 : {response}')

질문 : 김치는 어떤 음식인가요?
답변:김치는 한국의 대표적인 발효 음식으로, 배추에 고춧가루, 마늘, 생강 등을 넣어 만들어집니다.
질문 : 비빔밥을 어떻게 먹나요?
답변:비빔밥은 밥 위에 여러 가지 나물과 고기, 계란을 올린 후 고추장과 섞어 먹습니다.
질문 : 한국의 고기 요리에는 뭐가 있나요?
답변:한국의 고기 요리 중 하나로 불고기가 있습니다. 불고기는 양념한 소고기를 구워 먹는 전통 음식으로, 달콤하고 짭짤한 맛이 특징입니다.


## <span style="color: Gold"> **LLM 캐시**

In [ ]:
# 캐시없이 반복 호출 (동일한 질문을 반복하면) ---> OpenAI API 호출, 매번 모델에게 요청 --> 시간 오래 걸림, 비용 발생

import os
import time
from getpass import getpass
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()

questions = '대한민국의 수도는'

start = time.time()
for i in range(100):
    response = client.chat.completions.create(
                    model ='gpt-4o-mini',
                    messages= [{'role':'user', 'content':questions}],
                    temperature=0
    )
    answer = response.choices[0].message.content
elapsed_time = time.time() - start
print(f'elapsed_time : {elapsed_time}') 

elapsed_time : 69.46046209335327


In [ ]:
# 캐시
# 완전 일치 캐시 (Exact Match Cache)
# 본 코드 : 완전 일치 캐시 (Exact Match Cache)
# 동일한 입력 --> 저장된 응답 반환
# 장점 : 구현이 간단하고 100% 정확함
# 단점 : 완전히 같아야지만 작동

# 대한민국의 수도는?  캐시 히트
# 대한민국 수도는?    캐시 미스(다른 문자열)
# 한국의 수도는?      캐시 미스

cache = {}  # key --> questions , value --> answer 
response = []
start = time.time()

questions = '대한민국의 수도는'


for i in range(100):
    if questions in cache:  # 캐시히트 : 딕셔너리에 해당 key (questions)가 존재하는지 확인. 캐시에 이미 답이 있으면(캐시히트) API 호출 없이 바로 answer 가져오기
        answer = cache[questions]
    else:    # 캐시미스 : 캐시에 없으면 API 호출 후 답을 저장: cache[questions] = answer
        response = client.chat.completions.create(
                    model ='gpt-4o-mini',
                    messages= [{'role':'user', 'content':questions}],
                    temperature=0
        )
        answer = response.choices[0].message.content
        cache[questions] = answer

elapsed_time = time.time() - start
print(f'elapsed_time : {elapsed_time}') 



elapsed_time : 0.7181918621063232


In [ ]:
# 의미적 캐시 (Semantic Cache)
# 의미가 비슷한 입력 --> 저장된 응답 반환
# 1. 들어온 질문을 임베딩으로 변환
# 2. 벡터DB (벡터 스토어)에서 유사한(근접한) 기존 쿼리 벡터를 검색
# 3. 상위 결과의 유사도 점수가 사전 정의한 임계값(theshold)보다 크면, 그 항목의 저장된 응답을 사용
# 4. 없으면 LLM을 호출해 응답을 생성하고, 질문+응답을 임베딩해 벡터 DB에 추가

# 단순 문자열 비교가 아니라 임베딩(벡터) 기반 유사도(보통 코사인 유사도 사용)를 사용

# 장점 : 
# 높은 히트율
# 다양한 표현을 허용해서 유연함을 가짐
# 비용 절감

# 단점
# 약간 느림
# 벡터 DB 필요

In [ ]:
# %pip install chromadb

In [ ]:
# 벡터 스토어 ---> DB
# 문서나 텍스트를 벡터로 변환한 후에 저장 --> 유사도 기반 검색 기능

import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

import chromadb 
# 벡터데이터를 DB화함 : 로컬/인메모리/디스크에 벡터와 메타데이터를 저장하고 유사도 검색해 주는 벡터 DB 라이브러리.
# “텍스트·이미지·문서를 임베딩(벡터)로 변환해서 저장하고, 유사도 기반 검색을 할 수 있는 데이터베이스”



# 클라이언트 생성
client = chromadb.Client() 


# 컬렉션 생성
  # create_collection("my_collection")은 컬렉션(테이블 같은 개념)을 만든 것. 컬렉션은 문서·아이디·임베딩·메타데이터를 저장.
  # 벡터DB에서 문서(또는 청크)들을 저장하는 그릇
  # 단계1. 쿼리 --> 임베딩   //   단계2. 컬렉션 안에서 청크단위로 유사도 검색   // 단계3. 가장 비슷한 청크를 찾아 LLM에게 전달 -> 답생성 
collection = client.create_collection("my_collection")



# 문서와 임베딩 준비
texts = [
    '대한민국의 수도는 서울입니다.',
    '프랑스의 수도는 파리입니다.',
    '서울은 한국의 정치, 경제 중심지 입니다.'
]

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2') # sentence-transformers 라이브러리의 경량 임베딩 모델 all-MiniLM-L6-v2를 사용해 텍스트를 벡터(임베딩)로 변환.
embeddings = model.encode(texts).tolist()
embeddings


# 문서 추가
# collection.add로 컬렉션에 (id, document, embedding)을 저장. 이후 유사도 검색 시 이 임베딩들을 기준으로 탐색.
ids = ['doc1', 'doc2', 'doc3']
collection.add (
    ids=ids,
    documents=texts,
    embeddings=embeddings
)



# 유사도 검색
query = '조선의 수도는 어디인가요?'
query_embedding = model.encode([query]).tolist()    # SentenceTransformer 모델의 encode()는 기본적으로 Numpy 배열이나 PyTorch/Tensor 형태의벡터로 반환 --> 리스트로!
results = collection.query(
    query_embeddings=query_embedding,  # 벡터리스트 : 한개의 커리도 여러개 쿼리도 리스트안에 벡터 형태로 들어가있음
    n_results=1   # 가장 유사한 문서 몇개를 뽑을지
)
print(results)



c:\Users\playdata2\miniconda3\envs\OPENAI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'ids': [['doc1']], 'embeddings': None, 'documents': [['대한민국의 수도는 서울입니다.']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None]], 'distances': [[0.3832096457481384]]}


In [ ]:
# 상기 출력 결과 해석
# {
#   'ids': [['doc1']], =====================================> 검색결과에서 가장 유사한 문서의 아이디
#   'embeddings': None, ====================================> 결과에서는 벡터 자체를 반환하지 않았음
#   'documents': [['대한민국의 수도는 서울입니다.']], =========> 실제 검색된 문서 텍스트
#   'uris': None, ==========================================> 문서가 외부 URI로 연결된 경우 사용. 로컬 컬렉션이기 때문에 None
#   'included': ['metadatas', 'documents', 'distances'], ===> 어떤 정보가 검색결과에 포함되어 있는지 표시
#   'data': None, ==========================================> 부적으로 ChromaDB에서 관리하는 raw data. 기본쿼리에서는 필요없음
#   'metadatas': [[None]], =================================> 컬렉션에 추가할때 문서마다 metadata(카테고리, 출처 등)를 지정하지 않으면 None
#   'distances': [[0.3832096457481384]] ====================> 유사도 거리. 값이 낮을수록 벡터가 더 가까움/유사함
# }


##  <span style="color: Gold"> **다층 캐시 전략**
<span style="color: Gold"> 

- 메모리 (완전일치) - 미스 벡터DB(의미적) - 미스 LLM 호출

In [1]:
# 1. 완전 일치
class SimpleCache:

    # 생성자
    def __init__ (self):
        self.cache = {} # 딕셔너리 초기화 / 실제 캐시를 저장하는 공간
        self.hits = 0    # 캐시 적중(hit) 횟수
        self.misses = 0   # 캐시 실패(miss) 횟수

    # get 메서드 : 캐시에서 값을 꺼내오기
    def get (self,key):
        if key in self.cache:
            self.hits += 1
            return self.cache[key]
        self.misses += 1
        return None
    
    # set 메서드 : 질문과 답변을 캐시로 저장
    def set (self,key,value):
        self.cache[key] = value    # 질문이 key로 저장되고, 답변이 value로 저장

    
    # state 메서드 : 현재 캐시 상태 확인
    def state(self):
        total=self.hits + self.misses
        hit_rate = self.hits / total*100 if total > 0 else 0
        return {
            'hits' : self.hits,
            'misses' : self.misses,
            'hit_rate': hit_rate
        }

In [5]:
# 일치하지 않을때는 LLM을 사용하기 위해, 함수 만듬

import openai
from openai import OpenAI
client = OpenAI()
def call_llm(question):
  response = client.chat.completions.create(
      model = 'gpt-4o-mini',
      messages = [{'role':'user', 'content':question}],
      temperature = 0
    )
  return response.choices[0].message.content


In [ ]:
# 1. 완전 일치 캐시 함수 실행
cache = SimpleCache()
questions = [
    '대한민국의 수도는?',
    '대한민국의 수도는?',  # 캐시 히트
    '한국의 수도는?',      # 캐시 미스(다른 문자열)
]
for q in questions:
  cached = cache.get(q)
  if cached:
    print(f' 캐시 : {cached}')
  else:
    response = call_llm(q)
    cache.set(q,response)
    print(f' LLM : {response}')
print(cache.state())

 LLM : 대한민국의 수도는 서울입니다.
 캐시 : 대한민국의 수도는 서울입니다.
 LLM : 한국의 수도는 서울입니다.
{'hits': 1, 'misses': 2, 'hit_rate': 33.33333333333333}


In [9]:
# 2. 의미 기반 캐시 구현 (Semantic)
# 의미적 유사성 - 벡터 DB chromadb
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
# OpenAIEmbeddingFunction은 OpenAI의 임베딩 모델을 ChromaDB와 연결해주는 함수/클래스
# ChromaDB는 자체적으로 벡터를 생성하지 않아요. 따라서 텍스트 → 임베딩(벡터) 변환을 위해 OpenAI나 다른 임베딩 모델을 사용할 수 있어요.
# 여기서는 text-embedding-3-small 모델을 사용해 텍스트를 숫자 벡터로 변환합니다.
# 핵심: ChromaDB가 검색할 수 있도록 텍스트를 벡터로 바꿔주는 역할



import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
# client = OpenAI(api_key=os.environ['OPENAI_KEY'])
client = OpenAI() 


class SemanticCache:

    # 생성자 메서드
    def __init__(self, name = 'semantic_cache4'):
        # chromadb 클라이언트 생성, DB를 관리하는 객체
        self.client = chromadb.Client()
        
        # chromadb에서 임베딩을 생성할 떄 사용할 함수
        self.embed_fn = OpenAIEmbeddingFunction(  
            api_key= os.environ['OPENAI_API_KEY'],
            model_name="text-embedding-3-small"
        )

        # 컬렉션 생성
            # hnsw:space ==> hnsw = Hierarchical Navigable Small World graph 
            # 벡터 검색에서 **빠른 근사 최근접 탐색(Approximate Nearest Neighbor, ANN)**을 위해 많이 쓰이는 자료 구조
            # 대규모 벡터 DB에서 “가장 가까운 벡터”를 빠르게 찾는 알고리즘
            # space = 거리 계산 방법을 의미
            # cosine
            # 벡터 간 유사도를 계산할 때 cosine similarity 사용
            # Cosine similarity = 벡터 사이 각도를 계산하여 얼마나 방향이 비슷한지 측정
        self.collection = self.client.create_collection(
            name=name,
            embedding_function=self.embed_fn,
            metadata={'hnsw:space':'cosine'}
        )
    



    # get 메서드 : 캐시에서 의미상 유사한 답변이 있는지 검색
    # **거리(distance)**가 threshold보다 작으면 → 충분히 유사하다고 판단, 캐시에서 반환 # 없다면 → None 반환 (LLM 호출 필요)
     # threshold=0.2 유사도를 판단하는 기준 값 / 쿼리와 컬렉션 내 벡터간 거리가 이값보다 작으면 비슷하다고 판단/ 
     # 벡터의 거리가0.2이하라면 유사 -> 캐시 사용 / 0.2보다 크면 --> 다르다 --> LLM호출
    def get (self, query, threshold=0.2):  
        results = self.collection.query(
            query_texts = [query],  # 검색할 질문 텍스트
            n_results = 1   # 가장 유사한 1개 문서만 검색
        )
        print(f'get results : {results}')

        # [0] 단일 쿼리 기준으로 하나의 질문만 해당함. query_texts['질문1', '질문2'] 라면 [0]은 첫번째 질문만 처리. 여러질문에 대응되지는 않음.
        if results['distances'][0] and results['distances'][0][0] < threshold:
            return results['metadatas'][0][0]['response']

            # results['distances'] → 쿼리 벡터와 컬렉션 내 각 벡터 간 거리 리스트
            # 리스트 안에 리스트가 있는 이유: 멀티쿼리, 멀티결과 지원
            # results['distances'][0] → 첫 번째 쿼리의 모든 검색 결과 거리
            # results['distances'][0][0] → 첫 번째 쿼리에서 가장 유사한 문서의 거리
            # results['metadatas'] → 문서에 연결된 메타데이터(여기선 캐시된 LLM 응답)
            # results['metadatas'][0][0] → 첫 번째 쿼리, 가장 유사한 문서의 메타데이터

            # results['distances'][0] → 첫 번째 쿼리에 대한 검색 결과가 존재하는지 확인
            # results['distances'][0][0] < threshold → 가장 유사한 문서와의 거리(distance)가 threshold보다 작으면 → 충분히 비슷하다 판단
            # 즉, 검색된 문서 중에서 충분히 비슷한 문서가 있으면 캐시 사용이라는 의미입니다.
            
            # return : metadatas 안에 LLM의 이전 응답(response)이 저장되어 있으므로 가장 유사한 문서에 연결된 이전 답변을 반환
        return None




    # set 메서드 : 새 질문과 답변을 컬렉션(벡터 DB)에 저장함
    def set (self, query, response):  # 데이터베이스화..?
        import uuid  #unique id를 생성
        self.collection.add(
            documents=[query],  # 문자열 텍스트를 컬렉션에 추가하는 동작. self.collection 에서 embedding_function=self.embed_fn 지정되어 있으므로, 문서를 추가할 때 텍스트를 임베딩 벡터로 변환됨
            metadatas = [{'response' : response}],  # 답변은 메타데이터로 저장
            ids = [str(uuid.uuid4())]  # UUID로 고유 ID 생성
        )


In [4]:
import uuid
uuid.uuid4()

UUID('177a09a6-75f6-4c80-bd08-ff38e25173ac')

In [ ]:
# SemanticCache 사용
cache=SemanticCache(name='test2')

# 매번 다른 이름으로 캐시 생성
# import uuid
# cache = SemanticCache(name=f"semantic_cache_{uuid.uuid4()}")


In [ ]:
# SemanticCache 사용
questions = [
    '대한민국의 수도는?',
    '대한민국의 수도는?',  # 캐시 히트
    '한국의 수도는?',      # 캐시 미스(다른 문자열)
]
for q in questions:
  cached = cache.get(q)
  if cached:
    print(f'HIT : {q} - {cached}')
  else:
    response = call_llm(q)
    cache.set(q,response)
    print(f'MISS  : {q} - {response}')


get results : {'ids': [[]], 'embeddings': None, 'documents': [[]], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[]], 'distances': [[]]}
MISS  : 대한민국의 수도는? - 대한민국의 수도는 서울입니다.
get results : {'ids': [['e4750b8b-44bc-498e-a4b8-5ae20b0bf7a8']], 'embeddings': None, 'documents': [['대한민국의 수도는?']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'response': '대한민국의 수도는 서울입니다.'}]], 'distances': [[2.205371856689453e-06]]}
HIT : 대한민국의 수도는? - 대한민국의 수도는 서울입니다.
get results : {'ids': [['e4750b8b-44bc-498e-a4b8-5ae20b0bf7a8']], 'embeddings': None, 'documents': [['대한민국의 수도는?']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'response': '대한민국의 수도는 서울입니다.'}]], 'distances': [[0.19354510307312012]]}
HIT : 한국의 수도는? - 대한민국의 수도는 서울입니다.


- L1 메모리 내부 메모리 - dictionary
- L2 메모리 벡터 DB - 의미적 유사성
- L3 메모리 LLM호출

In [10]:
class MulltiLevelCache:
    def __init__(self) -> None :
        self.l1_cache =SimpleCache() # 메모리방식 dictionary 완전 일치
        self.l2_cache =SemanticCache() # chromaDB 벡터DB 유사도 방식
    
    def stats(self):
        print(f'L1 catch: {self.l1_cache.cache}')

    def get(self, key):
        cached = self.l1_cache.get(key)
        if cached:
            print('L1 캐시')
            return cached
        cached = self.l2_cache.get(key)
        if cached:
            print('L2 캐시')
            self.l1_cache.set(key,cached)
            return cached
        # LLM 호출
        print('LLM')
        response= call_llm(key)
        self.l1_cache.set(key, response) 
        self.l2_cache.set(key, response)
        return response

In [11]:
mlc = MulltiLevelCache()

In [ ]:
mlc.get('한국 수도는?')


# 실행 결과

# 질문1. '한국의 수도는'
# 첫번째 LLM '한국의 수도는 서울입니다.'
# 두번쨰 L1 캐시 '한국의 수도는 서울입니다.'

# 질문2. mlc.get('한국 수도는?')
# L2 캐시 '한국의 수도는 서울입니다.'

get results : {'ids': [['87916e4d-2437-4330-9a9b-9e6ad0436b7c']], 'embeddings': None, 'documents': [['한국의 수도는']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'response': '한국의 수도는 서울입니다.'}]], 'distances': [[0.1432930827140808]]}
L2 캐시


'한국의 수도는 서울입니다.'